In [1]:
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
import nltk

In [2]:
okt = Okt()

In [3]:
def load_data(file_path):
    # file_path에 있는 데이터를 읽어 옴

    train = []
    count = 0

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            if count == 500: break

            line = line.strip()
            id, doc, label = line.split('\t')

            if label == '1': label = 'pos'
            elif label == '0': label = 'neg'
            train.append((doc, label))

            count += 1

    return train

In [4]:
def pos_tokenize(raw_sent):
    pos_sent = []
    sent = okt.pos(raw_sent, norm=True, stem=True)

    for tup in sent:
        word, tag = tup[0], tup[1]
        word_tag = word + '/' + tag
        pos_sent.append(word_tag)

    return ' '.join(pos_sent)

In [5]:
def make_word_dict(train, use_morph=False):
    all_words = set()

    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)

        for word in words:
            all_words.add(word)

    return all_words

In [6]:
def make_train_feats(train, all_words, use_morph=False):
    train_features = []

    for tup in train:
        sent, label = tup[0], tup[1]
        if use_morph: sent = pos_tokenize(sent)
        words = word_tokenize(sent)

        tmp = {set_word: (set_word in words) for set_word in all_words}
        sent_tup = (tmp, label)
        train_features.append(sent_tup)

    return train_features

In [7]:
train = load_data('ratings_train.txt')

# 제대로 가져와졌는지 확인
# print(train[:5])

In [8]:
use_morph = True
all_words = make_word_dict(train, use_morph)
train_features = make_train_feats(train, all_words, use_morph)